# Exercise 4: Processing a Landsat Scene into Art

In this exercise, we will process a landsat scene directly from the raw GeoTiff, hosted on AWS. This is different then the Landsat Scene data we've been interacting with in the earlier exercises, which had been pre-processed by GeoTrellis into a GeoTrellis Layer.

After we grab process the landsat scene, we'll combine it with [NED](https://lta.cr.usgs.gov/NED) and [NLCD](https://catalog.data.gov/dataset/national-land-cover-database-nlcd-land-cover-collection) data and adjust coloring via [rio_color](https://github.com/mapbox/rio-color/tree/master/rio_color) to make a cool looking map layer.

In [ ]:
import rasterio
import osr
import rasterio.warp
import geopyspark as gps
import numpy as np
import csv, os
import matplotlib as mpl
import matplotlib.pyplot as plt
import io
import math
from PIL import Image
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform
from functools import partial
import urllib.request, json
import dateutil.parser

from datetime import datetime
from pyspark import SparkContext
from geonotebook.wrappers import TMSRasterData, GeoJsonData
from datetime import datetime
from collections import namedtuple

from rasterfoundry.api import API

In [ ]:
# Set up our spark context
conf = gps.geopyspark_conf(appName="Landsat") \
          .setMaster("local[*]") \
          .set(key='spark.ui.enabled', value='true') \
          .set(key="spark.driver.memory", value="8G")
sc = SparkContext(conf=conf)
"/home/hadoop/notebooks/exercises/exercise4/annotation-tool.png"

## Grab the GeoJson for our Area of Interest

We can use the annotation tool in GeoNotebook to grab an extent that we are interested in. The location of the tool is in the toolbar, highlighted here:
![Annotation tool](files/annotation-tool.png)

Draw a small bounding box in an area you'd like to processes a landsat scene for.

We can then use GeoNotebook to grab the annotation and get the polygon it represents.

In [ ]:
aoi = M.layers.annotation.rectangles[0]

In [ ]:
p = aoi.centroid
M.set_center(p.x, p.y, 9)

## Using Raster Foundry to find Landsast 8 scenes

Here we set up a client to read from the [Raster Foundry](https://www.rasterfoundry.com/) API any scenes that match our area of interest over a time in 2017.

In [ ]:
client = API(refresh_token='47MAq91iWa6xlbqEW5d6uustkslFI75ZaawBdzW2gVoZ0')
landsat_8 = client.get_datasources(name='Landsat 8').results[0]

min_datetime = datetime(2017, 1, 1).isoformat() + 'Z'
max_datetime = datetime(2017, 12, 1).isoformat() + 'Z'
bounds = ','.join(map(lambda x: str(x), aoi.bounds))

filters = dict(pageSize=250, datasource=[landsat_8.id],
                   minAcquisitionDatetime=min_datetime,
                   maxAcquisitionDatetime=max_datetime,
                   bbox=bounds,
                   maxCloudCover=10)

# Initial conditions
has_next = True
page = 0
results = []

while has_next:
    print("Processing Page {}".format(page))
    scenes = client.get_scenes(page=page, **filters)
    if page == 0:
        print('{} scenes total match query'.format(scenes.count))
    for scene in scenes.results:
        results.append(scene)
    page += 1
    has_next = scenes.hasNext

## Picking a scene

We can use the thumbnail information on the results to browse the scenes and select the one we want to work with.

In [ ]:
def plot_image(img, name):
    fig = plt.figure()
    fig.set_size_inches(6, 4)

    a = fig.add_subplot(1, 2, 1)
    a.set_title(name)
    plt.imshow(img)
    plt.show()

def get_thumbnail(scene, size='SMALL'):
    url = list(filter(lambda t: t.thumbnailSize == size, scene.thumbnails))[0].url
    file = io.BytesIO(urllib.request.urlopen(url).read())
    return Image.open(file)

In [ ]:
from matplotlib.pyplot import imshow

for i, scene in enumerate(results):
    plot_image(get_thumbnail(scene), "Scene {}".format(i))

In [ ]:
scene = results[5]
get_thumbnail(scene, size='LARGE')

## Create the RDD of Scene information

This bit of code grabs the relevant information from the Raster Foundry results for our scene, and parallizes that collection of information into an RDD.

In [ ]:
# Convenience tuple to store Scene information for GeoPySpark
SceneRow = namedtuple('SceneRow', 'date, scene_id, band, uri')

desired_bands = {
    # 'coastal aerosol - 1': 0,
    'blue - 2': 1,
    'green - 3': 2,
    'red - 4': 3,
    'near infrared - 5': 4,
    # 'swir - 6': 5,
    # 'swir - 7': 6,
    # 'panchromatic - 8': 7,
    # 'cirrus - 9': 8,
    # 'thermal infrared - 10': 9,
    # 'thermal infrared - 11': 10,
    #'QA': 11
}

def get_desired_bands(scene):
    """Convenience function to process desired bands for GeoPySpark

    Args:
        scene:

    Returns:
        List[SceneRow]
    """
    acquisition_date = scene.filterFields.acquisitionDate
    landsat_8_rows = []
    for image in scene.images:
        uri = image.sourceUri
        band_name = image.bands[0].name
        band_num = desired_bands.get(band_name)
        if band_num:
            row = SceneRow(acquisition_date, scene.name, band_num, uri)
            landsat_8_rows.append(row)

    return landsat_8_rows

In [ ]:
scene_info = sc.parallelize(get_desired_bands(scene)).repartition(4)

In [ ]:
def get_metadata(row):
    with rasterio.open(row.uri) as dataset:
        bounds = dataset.bounds
        height = dataset.height
        width = dataset.width
        crs = dataset.get_crs()
        srs = osr.SpatialReference()
        srs.ImportFromWkt(crs.wkt)
        proj4 = srs.ExportToProj4()
        tile_cols = math.floor((width - 1) / 512) * 512
        tile_rows = math.floor((height - 1) / 512) * 512
        ws = [((x, x + 512), (y, y + 512)) for x in range(0,tile_cols, 512) \
                                          for y in range(0, tile_rows, 512)]

    def windows(row, ws):
        for w in ws:
            ((row_start, row_stop), (col_start, col_stop)) = w

            left  = bounds.left + (bounds.right - bounds.left)*(float(col_start)/width)
            right = bounds.left + (bounds.right - bounds.left)*(float(col_stop)/ width)
            bottom = bounds.top + (bounds.bottom - bounds.top)*(float(row_stop)/height)
            top = bounds.top + (bounds.bottom - bounds.top)*(float(row_start)/height)
            extent = gps.Extent(left,bottom,right,top)
            instant = datetime.strptime(row.date, "%Y-%m-%dT%H:%M:%SZ")

            projected_extent = gps.TemporalProjectedExtent(extent=extent, 
                                                            instant=instant, 
                                                            proj4=proj4)
            window_info = { 'scene_id': row.scene_id, 
                            'band': row.band, 
                            'uri': row.uri,
                            'window': w,
                            'projected_extent': projected_extent,
                            'tile_key': (row_start, col_start) }

            yield window_info
    
    return [i for i in windows(row, ws)]

scene_window_metadata = scene_info.flatMap(get_metadata)


In [ ]:
def get_data(line):
    
    new_line = line.copy()

    with rasterio.open(line['uri']) as dataset:
        new_line['data'] = dataset.read(1, window=line['window'])
        new_line.pop('window')
        new_line.pop('uri')
    
    return new_line


In [ ]:
scene_band_tiles = scene_window_metadata.repartition(500).map(get_data)

In [ ]:
grouped_band_tiles = \
  scene_band_tiles.groupBy(lambda line: (line['scene_id'], line['tile_key']))

In [ ]:
def make_tiles(grouped):
    lines = list(grouped[1])
    projected_extent = lines[0]['projected_extent']
    try:
        array = np.array([l['data'] for l in sorted(lines, key=lambda l: int(l['band']))])
        if array.dtype == 'object':
            bandshapes = [str(l['data'].shape) for l in sorted(lines, key=lambda l: l['band'])]
            raise Exception("{}".format('\n'.join(bandshapes)))
    except:
        bandshapes = ["{} - {}".format(l['band'], l['data'].shape) for l in sorted(lines, key=lambda l: l['band'])]
        raise Exception("ER {}".format('\n'.join(bandshapes)))
    tile = gps.Tile.from_numpy_array(array, no_data_value=0.0)
    return (projected_extent, tile)

In [ ]:
combined_tiles = grouped_band_tiles.map(make_tiles)

In [ ]:
raster_layer = gps.RasterLayer.from_numpy_rdd(gps.LayerType.SPACETIME, combined_tiles)

In [ ]:
tiled_raster_layer = raster_layer.tile_to_layout(layout = gps.GlobalLayout(), target_crs=3857)

In [ ]:
landsat_layer = tiled_raster_layer.to_spatial_layer()

In [ ]:
landsat_pyramid = landsat_layer \
                    .repartition(100) \
                    .pyramid(resample_method=gps.ResampleMethod.BILINEAR)

In [ ]:
# If we wanted to write the pyramid, we could do so like this:
"""
for layer in sorted(pyramid.levels.values(), key=lambda l: -l.zoom_level):
    gps.write("s3://some/catalog", "my-landsat-image", layer, time_unit=gps.TimeUnit.DAYS)
"""

In [ ]:
def landsat_rgba(tile):
    cells = tile.cells
    # Color correct - use magic numbers
    magic_min, magic_max = 4000, 15176
    norm_range = magic_max - magic_min
    cells = cells.astype('int32')
    # Clamp cells
    cells[(cells != 0) & (cells < magic_min)] = magic_min
    cells[(cells != 0) & (cells > magic_max)] = magic_max
    colored = ((cells - magic_min) * 255) / norm_range
    r, g, b = (colored[2], colored[1], colored[0])
    alpha = np.full(r.shape, 255)
    alpha[(cells[0] == tile.no_data_value) & \
          (cells[1] == tile.no_data_value) & \
          (cells[2] == tile.no_data_value)] = 0
    return (r, g, b, alpha)

In [ ]:
def render_image(tile):
    (r, g, b, alpha) = landsat_rgba(tile)
    rgba = np.dstack([r,g,b, alpha]).astype('uint8')
    return Image.fromarray(rgba, mode='RGBA')

for x in M.layers:
    M.remove_layer(x)

tms_server = gps.TMS.build(landsat_pyramid, display=render_image)
M.add_layer(TMSRasterData(tms_server), name="landsat")

# Add in hillshade from National Elevation Dataset

In [ ]:
elevation_layer = gps.query("s3://azavea-datahub/catalog", 
                            "us-ned-tms-epsg3857", 
                            layer_zoom=13,
                            query_geom=landsat_layer.layer_metadata.extent.to_polygon,
                            num_partitions=500)

In [ ]:
hillshade = gps.hillshade(elevation_layer, band=0, azimuth=99.0, altitude=33.0, z_factor=0.0)


In [ ]:
hillshade_pyramid = hillshade.repartition(100).pyramid(resample_method=gps.ResampleMethod.BILINEAR)

In [ ]:
# See http://fwarmerdam.blogspot.com/2010/01/hsvmergepy.html
# =============================================================================
# rgb_to_hsv()
#
# rgb comes in as [r,g,b] with values in the range [0,255].  The returned
# hsv values will be with hue and saturation in the range [0,1] and value
# in the range [0,255]
#
def rgb_to_hsv( r,g,b ):

    maxc = np.maximum(r,np.maximum(g,b))
    minc = np.minimum(r,np.minimum(g,b))

    v = maxc

    minc_eq_maxc = np.equal(minc,maxc)

    # compute the difference, but reset zeros to ones to avoid divide by zeros later.
    ones = np.ones((r.shape[0],r.shape[1]))
    maxc_minus_minc = np.choose( minc_eq_maxc, (maxc-minc,ones) )

    s = (maxc-minc) / np.maximum(ones,maxc)
    rc = (maxc-r) / maxc_minus_minc
    gc = (maxc-g) / maxc_minus_minc
    bc = (maxc-b) / maxc_minus_minc

    maxc_is_r = np.equal(maxc,r)
    maxc_is_g = np.equal(maxc,g)
    maxc_is_b = np.equal(maxc,b)

    h = np.zeros((r.shape[0],r.shape[1]))
    h = np.choose( maxc_is_b, (h,4.0+gc-rc) )
    h = np.choose( maxc_is_g, (h,2.0+rc-bc) )
    h = np.choose( maxc_is_r, (h,bc-gc) )

    h = np.mod(h/6.0,1.0)

    hsv = np.asarray([h,s,v])

    return hsv

# =============================================================================
# hsv_to_rgb()
#
# hsv comes in as [h,s,v] with hue and saturation in the range [0,1],
# but value in the range [0,255].

def hsv_to_rgb( hsv ):

    h = hsv[0]
    s = hsv[1]
    v = hsv[2]

    #if s == 0.0: return v, v, v
    i = (h*6.0).astype(int)
    f = (h*6.0) - i
    p = v*(1.0 - s)
    q = v*(1.0 - s*f)
    t = v*(1.0 - s*(1.0-f))

    r = i.choose( v, q, p, p, t, v )
    g = i.choose( t, v, v, q, p, p )
    b = i.choose( p, p, t, v, v, q )

    return (r,g,b)

In [ ]:
def render_with_hillshade(tiles):
    landsat_tile = tiles[0]
    hillshade_tile = tiles[1]
    (r, g, b, alpha) = landsat_rgba(tiles[0])
    rgba = np.dstack([r,g,b, alpha]).astype('uint8')
    hsv = rgb_to_hsv(r, g, b)
    z = hillshade_tile.cells[0]
    z = (z * 256) / 200
    z = (z * 2 + hsv[2]) / 3
    hsv[2] = z
    (r, g, b) = hsv_to_rgb(hsv)

    rgba = np.dstack([r,g,b,alpha]).astype('uint8')
    return Image.fromarray(rgba, mode='RGBA')

    

In [ ]:
for x in M.layers:
    M.remove_layer(x)

tms_server = gps.TMS.build([landsat_pyramid, hillshade_pyramid], display=render_with_hillshade)
M.add_layer(TMSRasterData(tms_server), name="landsat")

## Add in NLCD

In [ ]:
nlcd_layer = gps.query("s3://datahub-catalogs-us-east-1", 
                      "nlcd-zoomed-256", 
                      layer_zoom=13,
                      query_geom=landsat_layer.layer_metadata.extent.to_polygon,
                      num_partitions=100)

labels = { 0: 'NoData',
          11: 'Open Water',
          12: 'Perennial Ice/Snow',
          21: 'Developed, Open Space',
          22: 'Developed, Low Intensity',
          23: 'Developed, Medium Intensity',
          24: 'Developed High Intensity',
          31: 'Barren Land (Rock/Sand/Clay)',
          41: 'Deciduous Forest',
          42: 'Evergreen Forest ',
          43: 'Mixed Forest',
          52: 'Shrub/Scrub',
          71: 'Grassland/Herbaceous',
          81: 'Pasture/Hay',
          82: 'Cultivated Crops',
          90: 'Woody Wetlands',
          95: 'Emergent Herbaceous Wetlands'}

In [ ]:
nlcd_pyramid = nlcd_layer.repartition(100).pyramid()

In [ ]:
def render3(tiles):
    landsat_tile = tiles[0]
    hillshade_tile = tiles[1]
    nlcd_tile = tiles[2]
    (r, g, b, alpha) = landsat_rgba(tiles[0])
    #rgba = np.dstack([r,g,b, alpha]).astype('uint8')
    #return Image.fromarray(rgba, mode='RGBA')
    hsv = rgb_to_hsv(r, g, b)
    z = hillshade_tile.cells[0]
    z = (z * 256) / 200
    z = (z * 3 + hsv[2]) / 4
    hsv[2] = z
    (h_r, h_g, h_b) = hsv_to_rgb(hsv)
    
    nlcd = nlcd_tile.cells[0]
    # Copy original values in developed land
    developed_land = (nlcd >= 22) & (nlcd <= 23)
    np.copyto(h_r, r, where=developed_land)
    np.copyto(h_g, r, where=developed_land)
    np.copyto(h_b, r, where=developed_land)
    
    ### rio_color color correction
    
    # Move rgb values to 0.0 - 1.0 space
    rgb = np.array([h_r, h_g, h_b])
    rgb = rgb.astype(float)
    rgb[rgb < 0] = 0
    rgb /= 256.0

    # Adjust gamma and sigmoidal contrast
    rgb = gamma(rgb, 1.2)
    rgb = sigmoidal(rgb, 3, 0.45)

    # Saturate water
    np.copyto(rgb,  saturation(rgb, 2), where=nlcd == 11)
    
    # Convert back to byte space
    rgb *= 256

    # Set water opacity to 80%
    np.copyto(alpha, (alpha * 0.8).astype('int64'), where=nlcd == 11)
    
    rgba = np.dstack([rgb[0],rgb[1],rgb[2],alpha]).astype('uint8')
    return Image.fromarray(rgba, mode='RGBA')

In [ ]:
for x in M.layers:
    M.remove_layer(x)
    
tms_server = gps.TMS.build([landsat_pyramid, hillshade_pyramid, nlcd_pyramid], display=render3)
M.add_layer(TMSRasterData(tms_server), name="landsat")

## Add in NDVI

In [ ]:
def safe_divide(a, b):
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide(a, b)
        c[c == np.inf] = 0
        c = np.nan_to_num(c)
        return c


def compute_ndvi(tile):
    cells = tile.cells.astype(float)
    red = cells[2]
    ir = cells[3]
    return  safe_divide((ir - red), (ir + red))

In [ ]:
from rio_color.operations import gamma, sigmoidal, saturation

def render3(tiles):
    landsat_tile = tiles[0]
    hillshade_tile = tiles[1]
    nlcd_tile = tiles[2]
    ndvi = compute_ndvi(landsat_tile)
    nlcd = nlcd_tile.cells[0]

    (r, g, b, alpha) = landsat_rgba(tiles[0])
 
    hsv = rgb_to_hsv(r, g, b)
    z = hillshade_tile.cells[0]
    z = (z * 256) / 200
    z = (z * 3 + hsv[2]) / 4
    hsv[2] = z
    (h_r, h_g, h_b) = hsv_to_rgb(hsv)
    
   
    # Copy original values in developed land
    developed_land = (nlcd >= 22) & (nlcd <= 23)
    np.copyto(h_r, r, where=developed_land)
    np.copyto(h_g, r, where=developed_land)
    np.copyto(h_b, r, where=developed_land)
    
    ### rio_color color correction
    
    # Move rgb values to 0.0 - 1.0 space
    rgb = np.array([h_r, h_g, h_b])
    rgb = rgb.astype(float)
    rgb[rgb < 0] = 0
    rgb /= 256.0

    # Adjust gamma and sigmoidal contrast
    rgb = gamma(rgb, 1.2)
    rgb = sigmoidal(rgb, 3, 0.45)
    
    # Saturate water
    np.copyto(rgb,  saturation(rgb, 2), where=nlcd == 11)
    
    g = rgb[1]
    # Higher brightness for NDVI
    np.copyto(g, gamma(g, 1.2), where=ndvi > 0.3)
    np.copyto(g, gamma(g, 1.4), where=ndvi > 0.6)
    np.copyto(g, gamma(g, 1.7), where=ndvi > 0.9)
    
    # Higher contrast for NDVI
    np.copyto(g, sigmoidal(g, 3, 0.45), where=ndvi > 0.3)
    np.copyto(g, sigmoidal(g, 4, 0.4), where=ndvi > 0.6)
    np.copyto(g, sigmoidal(g, 5, 0.35), where=ndvi > 0.9)
    
    rgb = np.array([rgb[0], g, rgb[2]])
    
    # Saturate high NDVI
    np.copyto(rgb, saturation(rgb, 0.5), where=ndvi > 0.1)
    np.copyto(rgb, saturation(rgb, 0.8), where=ndvi > 0.2)
    np.copyto(rgb, saturation(rgb, 1.1), where=ndvi > 0.45)
    
    # Convert back to byte space
    rgb *= 256

    # Set water opacity to 80%
    np.copyto(alpha, (alpha * 0.8).astype('int64'), where=nlcd == 11)
    
    rgba = np.dstack([rgb[0],rgb[1],rgb[2],alpha]).astype('uint8')
    return Image.fromarray(rgba, mode='RGBA')


In [ ]:
for x in M.layers:
    M.remove_layer(x)

tms_server = gps.TMS.build([landsat_pyramid, hillshade_pyramid, nlcd_pyramid], display=render3)
M.add_layer(TMSRasterData(tms_server), name="landsat")